**[아직 정리하지 않은 문서입니다. 추후 업데이트 예정]**

## Semantic ranking in Azure AI Search using Python

Azure AI Search supports secondary L2 ranking that rescores initial results using machine reading comprehension models. The L2 ranker promotes more semantically relevant matches to the top.

This is the source code for the article: [Quickstart: Semantic ranking (Python)](https://learn.microsoft.com/azure/search/search-get-started-semantic).

## Install packages and set variables

In [ ]:
! pip install -r requirements.txt --quiet

In [ ]:
# Provide variables
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # Take environment variables from .env.

# The following variables from your .env file are used in this notebook
search_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "hotels-sample-index")

In [ ]:
import requests
from azure.core.exceptions import HttpResponseError

print("=" * 60)
print("🔍 AI Search 접근 진단 시작")
print("=" * 60)

# 1. 환경 변수 확인
print("\n1️⃣ 환경 변수 확인:")
print(f"   Search Endpoint: {search_endpoint}")
print(f"   Index Name: {index_name}")

# 2. 인증 정보 확인
print("\n2️⃣ 인증 정보 확인:")
try:
    # DefaultAzureCredential이 사용 가능한 credential 확인
    token = credential.get_token("https://search.azure.com/.default")
    print("   ✅ DefaultAzureCredential 토큰 획득 성공")
    print(f"   토큰 만료 시간: {token.expires_on}")
except Exception as ex:
    print(f"   ❌ DefaultAzureCredential 토큰 획득 실패: {ex}")
    print("   해결: az login 명령으로 Azure CLI 로그인 필요")

# 3. AI Search 서비스 접근 테스트
print("\n3️⃣ AI Search 서비스 접근 테스트:")
from azure.search.documents.indexes import SearchIndexClient

try:
    index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
    
    # 인덱스 목록 조회 시도
    print("   인덱스 목록 조회 중...")
    indexes = list(index_client.list_indexes())
    print(f"   ✅ 접근 성공! 총 {len(indexes)}개 인덱스 발견")
    for idx in indexes:
        print(f"      - {idx.name}")
    
except HttpResponseError as ex:
    print(f"   ❌ HTTP 에러 발생: {ex.status_code} - {ex.reason}")
    print(f"   에러 메시지: {ex.message}")
    
    if ex.status_code == 403:
        print("\n   🔧 Forbidden (403) 에러 해결 방법:")
        print("   1. Azure Portal에서 AI Search 리소스 확인")
        print("   2. '액세스 제어(IAM)' > '역할 할당' 메뉴로 이동")
        print("   3. 다음 역할 중 하나를 본인 계정에 할당:")
        print("      - Search Service Contributor (검색 인덱스 관리)")
        print("      - Search Index Data Contributor (데이터 읽기/쓰기)")
        print("      - Search Index Data Reader (데이터 읽기 전용)")
        print("   4. 네트워크 설정 확인:")
        print("      - '네트워킹' 메뉴에서 '모든 네트워크' 허용 여부 확인")
        print("      - 방화벽 규칙으로 IP가 차단되었는지 확인")
        print("\n   5. 현재 로그인 계정 확인:")
        print("      - PowerShell에서 'az account show' 실행")
        print("      - 올바른 구독/테넌트에 로그인되어 있는지 확인")
        
except Exception as ex:
    print(f"   ❌ 예상치 못한 에러: {type(ex).__name__}")
    print(f"   {str(ex)}")

print("\n" + "=" * 60)
print("진단 완료")
print("=" * 60)

## Get the index definition

This code retrieves the hotels-sample-index and checks for an existing semantic configuration. 

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.identity import DefaultAzureCredential
import os

# Initialize the client (similar to what you already have)
search_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = "hotels-sample-index"  # or use your existing index_name variable

# Create the SearchIndexClient
index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)

try:
    # Get the existing index schema
    index = index_client.get_index(index_name)
    
    print(f"Index name: {index.name}")
    print(f"Number of fields: {len(index.fields)}")
    
    # Print field details
    for field in index.fields:
        print(f"Field: {field.name}, Type: {field.type}, Searchable: {field.searchable}")
    
    # Access semantic configuration if it exists
    if index.semantic_search and index.semantic_search.configurations:
        for config in index.semantic_search.configurations:
            print(f"Semantic config: {config.name}")
            if config.prioritized_fields.title_field:
                print(f"Title field: {config.prioritized_fields.title_field.field_name}")
    else:
        print("No semantic configuration exists for this index")

except Exception as ex:
    print(f"Error retrieving index: {ex}")

## Add a semantic configuration to the index

This code adds a semantic configuration to an existing hotels-sample-index on your search service. No search documents are deleted by this operation and your index is still operational after the configuration is added.

In [ ]:
# Add semantic configuration to hotels-sample-index and display updated index details
from azure.search.documents.indexes.models import (
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)

try:
    # Get the existing index
    existing_index = index_client.get_index(index_name)
    
    # Create a new semantic configuration
    new_semantic_config = SemanticConfiguration(
        name="semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="HotelName"),
            keywords_fields=[SemanticField(field_name="Tags")],
            content_fields=[SemanticField(field_name="Description")]
        )
    )
    
    # Add semantic configuration to the index
    if existing_index.semantic_search is None:
        existing_index.semantic_search = SemanticSearch(configurations=[new_semantic_config])
    else:
        # Check if configuration already exists
        config_exists = any(config.name == "semantic-config" 
                          for config in existing_index.semantic_search.configurations)
        if not config_exists:
            existing_index.semantic_search.configurations.append(new_semantic_config)
    
    # Update the index
    result = index_client.create_or_update_index(existing_index)
    
    # Get the updated index and display detailed information
    updated_index = index_client.get_index(index_name)
    
    print("Semantic configurations:")
    print("-" * 40)
    if updated_index.semantic_search and updated_index.semantic_search.configurations:
        for config in updated_index.semantic_search.configurations:
            print(f"  Configuration: {config.name}")
            if config.prioritized_fields.title_field:
                print(f"    Title field: {config.prioritized_fields.title_field.field_name}")
            if config.prioritized_fields.keywords_fields:
                keywords = [kf.field_name for kf in config.prioritized_fields.keywords_fields]
                print(f"    Keywords fields: {', '.join(keywords)}")
            if config.prioritized_fields.content_fields:
                content = [cf.field_name for cf in config.prioritized_fields.content_fields]
                print(f"    Content fields: {', '.join(content)}")
            print()
    else:
        print("  No semantic configurations found")
    
    print("✅ Semantic configuration successfully added!")
    
except Exception as ex:
    print(f"❌ Error adding semantic configuration: {ex}")

## Run a term query

Your first query is a keyword search, specified by `query_type` set to simple. For comparison, run this query to review the results when no semantic ranking is used. This query has search text, which provides criteria for potential matches. Output should consist of 13 documents containing literal or variant instances of the search terms.

In [ ]:
from azure.search.documents import SearchClient

# Create the SearchIClient
search_client = SearchClient(endpoint=search_endpoint, credential=credential, index_name=index_name)

# Run a text query (returns a BM25-scored result set)
results =  search_client.search(query_type='simple',
    search_text="walking distance to live music" ,
    select='HotelId,HotelName,Description',
    include_total_count=True)

for result in results:
    print(result["@search.score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

## Run a semantic query

This example sets `query_type` to semantic. Compared to the previous query, the L2 ranker revisits the search results and rescores them using machine reading comprehension models, promoting results that are more meaningful to the query. Output should consist of 13 documents in a revised ranking. The effect of semantic ranking is apparent in the top half of the results, where results in positions four through six are more semantically aligned with the search text.

In [ ]:
# Runs a semantic query (runs a BM25-ranked query and promotes the most relevant matches to the top)
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
    search_text="walking distance to live music", 
    select='HotelId,HotelName,Description', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

## Return captions

A semantic query can return captions, or a portion of the matching text. Captions are useful if content is verbose or if you want hit highlights in the results.

In shorter descriptions like the ones in this sample index, captions are less useful and you might consider omitting them if they don't add value to your solution.

In [ ]:
# Runs a semantic query that returns captions
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
    search_text="walking distance to live music", 
    select='HotelId,HotelName,Description', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

## Return semantic answers

A semantic query response can also includes "answers". This capability is useful if your index contains verbatim content that has the characteristics of a complete answer. To get an answer, the query must have the characteristics of a question, such as "what is" or "when is".

To meet the requirements of semantic answers, provide a query that asks a question.

Recall that answers are verbatim content pulled from your index. To get composed answers as generated by a chat completion model, considering using a [RAG pattern](https://learn.microsoft.com/azure/search/retrieval-augmented-generation-overview) or [agentic retrieval](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept).

In [ ]:
# Run a semantic query that returns semantic answers  
results =  search_client.search(query_type='semantic', semantic_configuration_name='semantic-config',
 search_text="what's a good hotel for people who like to read",
 select='HotelId,HotelName,Description', query_caption='extractive', query_answer="extractive",)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(result["@search.reranker_score"])
    print(result["HotelId"])
    print(result["HotelName"])
    print(f"Description: {result['Description']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

## Clean up

If you're finished with this index, you can delete it by running the following lines. Deleting unnecessary indexes frees up space for stepping through more quickstarts and tutorials.

In [ ]:
try:
    result = index_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Confirm the index deletion by running the following script that returns the index definition. If hotels-sample-index isn't returned, you've successfully deleted the index and have completed this quickstart.

In [ ]:
try:
    result = index_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)
